### Imports

In [6]:
import pandas as pd
import numpy as np
#from scipy import 
#from sklearn import
from datetime import datetime, timedelta
import re
import datetime
import pywt
# pd.set_option("display.max_rows", 100)
# from IPython.core.display import display
import matplotlib.pyplot as plt

### General variables

In [7]:
data_folder = '../../Data/'
print_text_result = False
raw_data_names = ['pit1_data-2022', 'pit2_data-2022', 'pit3_data-2022', 'pit4_data-2022',
                  'pit1_data-2023', 'pit2_data-2023', 'pit3_data-2023', 'pit4_data-2023']
clean_data_names = ['VII_PIT1_2022', 'VII_PIT2_2022', 'VII_PIT3_2022', 'VII_PIT4_2022']
df_pairs = [('pit1_data-2022', 'VII_PIT1_2022'), ('pit2_data-2022', 'VII_PIT2_2022'), ('pit3_data-2022', 'VII_PIT3_2022'), ('pit4_data-2022', 'VII_PIT4_2022'),
            ('pit1_data-2023', None), ('pit2_data-2023', None), ('pit3_data-2023', None), ('pit4_data-2023', None)]
col_types = ['int64', 'int64', 'int64', 'int64', 'int64', 'float64', 'int64', 'float64', 'int64', 'float64',
             'int64', 'float64', 'int64', 'float64', 'int64', 'float64', 'float64', 'int64', 'float64', 'float64',
             'float64', 'float64', 'int64', 'float64', 'float64', 'float64', 'float64', 'float64', 'bool']

### Helper functions

In [8]:

def print_result(header: str, init_count: int, new_count: int):
    print(header)
    print(f'\tinit row cnt: {init_count}')
    print(f'\t# of rows deleted: {init_count - new_count}')
    print(f'\tresult row count : {new_count}')

def remove_pit_suffix(name: str) -> str:
    """
    Remove suffix '_pit<number>' from header
    """
    re_match = re.search(r'_pit\d+$', name)
    if re_match:
        name = name[:re_match.start()]
    return name

def prune_unnecessary_features(df: pd.DataFrame):
    unnecessary_features = ['Temp_T21_Avg(1)','Temp_T21_Avg(2)', 'Temp_T21_Avg(3)', 'Temp_T21_Avg(4)', 'Temp_T21_Avg(5)',
                            'CCVWC_Avg(1)', 'CCVWC_Avg(2)', 'CCVWC_Avg(3)', 'CCVWC_Avg(4)', 'CCVWC_Avg(5)', 'shf_plate_Avg',
                            'shf_multiplier', 'shf_htr_resstnc', 'shfp_wrnng_flg', 'btt_wrnng_flg', 'PTemp_C_Avg', 'RECORD', 'BattV_Min']
    df.drop(unnecessary_features, axis=1, inplace=True)

def filter_dates(df: pd.DataFrame, file_name: str) -> int:
    init_row_cnt = len(df)
    start_date_2023 = datetime.datetime(2023, 1,1, 0, 0, 0)

    if '2022' in file_name:
        df.drop(df[df['TIMESTAMP'] >= start_date_2023].index, inplace=True)
    else:
        df.drop(df[df['TIMESTAMP'] < start_date_2023].index, inplace=True)

    if print_text_result: print_result('FILTERING DATES BY YEAR:', init_row_cnt, len(df))

    return (init_row_cnt - len(df))

def filter_duplicate_date(df: pd.DataFrame) -> int:
    init_row_cnt = len(df)
    
    df.drop_duplicates(subset='TIMESTAMP', inplace=True)

    if print_text_result: print_result('FILTERING DUPLICATE DATES:', init_row_cnt, len(df))

    return (init_row_cnt - len(df))

def calculate_wavelet_coefficients(df: pd.DataFrame, period_lower_bound: float, period_upper_bound: float, file_name: str) -> None:
    
    # period of the wave T is calculated in days, frequency = 1/T 
    dt = (datetime.datetime(2023,1,1,0,5,0) - datetime.datetime(2023,1,1,0,0,0)).seconds / (60 * 60 * 24)

    if 'VII' in file_name:
        return df

    for sensor in np.arange(5):
        redox_series = "Redox_Avg(" + str(sensor + 1) + ")"     
        scales = np.geomspace(1, 2400, 30)
        signal = df[redox_series]
        [coefficients, frequencies] = pywt.cwt(signal, scales, "cmor1.5-0.5", dt)
        power = abs(coefficients)
        periods = 1 / frequencies
        coef_idx = np.where((periods >= period_lower_bound) & (periods <= period_upper_bound), True, False)
        power = power[np.arange(len(frequencies))[coef_idx], :].T
        wavelet_cols = ["Wave_period_" + str(round(period,1)) + "(" + str(sensor + 1) + ")" for period in periods[np.arange(len(frequencies))[coef_idx]]]
        wavelet_df = pd.DataFrame(power, columns = wavelet_cols, index = df.index)
        df = pd.concat([df, wavelet_df], axis = 1)
    
    return df

def fill_na_redox_values(df: pd.DataFrame, file_name: str) -> None:
    
    if 'VII' in file_name:
        return df

    for sensor in np.arange(5):
        redox_series = "Redox_Avg(" + str(sensor + 1) + ")"
        df[redox_series] = df[redox_series].ffill()
        
    return df

def filter_missing_values(df: pd.DataFrame, file_name: str, remove_empy=True) -> int:
    init_row_cnt = df.shape[0]
    if remove_empy and 'VII' not in file_name:
        df.dropna(inplace=True)
    # TODO else:
        # filter cleaned data
        # fill missing data

    if print_text_result: print_result('FILTERING MISSING VALUES:', init_row_cnt, len(df))

    return (init_row_cnt - len(df))

def filter_df(df: pd.DataFrame, file_name: str) -> dict:
    """
    Filters the given DataFrame by removing specific rows
    """
    diff_dict = dict()
    prune_unnecessary_features(df)
    diff_dict['dates_removed'] = filter_dates(df, file_name)
    diff_dict['duplicates_removed'] = filter_duplicate_date(df)
    df = calculate_wavelet_coefficients(df, 1/2, 5, file_name)
    diff_dict['missing_data_removed'] = filter_missing_values(df, file_name)
    return diff_dict

def fix_types(df: pd.DataFrame, file_name: str) -> pd.DataFrame:
    """
    Fix data types from given DataFrame
    """
    if 'VII' in file_name:
        return df
    for col_type, col_name in zip(col_types, list(df.columns.array)[1:]):
        if col_type == 'int64':
            df[col_name] = df[col_name].astype('float64').astype('int64')
        elif col_type == 'float64':
            df[col_name] = df[col_name].astype('float64')
        else:
            df[col_name] = df[col_name].astype('bool')
    return df

def add_pit_column(df: pd.DataFrame, file_name: str) -> pd.DataFrame:
    if 'VII' in file_name:
        return df
    pit_n = file_name[3]
    df['pit_number'] = int(pit_n)
    return df

def get_time_windows(hour_prediods: list[int]):
    return [int((period_h*60)/5) for period_h in hour_prediods]

def timestamp_gap(df: pd.DataFrame, td: timedelta, print_stats_graphs: bool = True) -> tuple: 
    df["TIMESTAMP_DIFF"] = df.loc[:, "TIMESTAMP"].diff()
    
    breaks = list(df.index[(df["TIMESTAMP_DIFF"] != td) & (np.isnan(df["TIMESTAMP_DIFF"]) == False)])
    gaps = [int(df.loc[index, "TIMESTAMP_DIFF"].total_seconds()/60) for index in breaks]
    
    gaps_observations = pd.DataFrame([[index, gap] for index, gap in zip(breaks, gaps)])
    if gaps_observations.empty == False:
        gaps_observations.columns = ["Observation index", "Time delta (min)"]
        
        if print_stats_graphs == True:
            print(f"the observations having gaps")
            print(gaps_observations)
    else: 
        if print_stats_graphs == True:
            print(f"there are no gaps")
    return (breaks, gaps)

def calculate_backward_sigma(rolling_window: pd.DataFrame, breaks: list, window_size: int, starting_index: int):
    if rolling_window.index[-1] < starting_index + window_size-1:
        return([np.nan, np.nan, np.nan, np.nan, np.nan])
    for j in breaks:
        break_zero_index = rolling_window.index[-1] - j
        if (break_zero_index >=0) and (break_zero_index < window_size-1):
            return([np.nan, np.nan, np.nan, np.nan, np.nan])
    return rolling_window[["Redox_Avg(1)", "Redox_Avg(2)", "Redox_Avg(3)", "Redox_Avg(4)", "Redox_Avg(5)"]].apply(func=np.std, axis=0)

def sigma_feature_engineering(df: pd.DataFrame, window_sizes, td, print_sigma_statistics = False):
    breaks = timestamp_gap(df, td)[0]

    for window_size in window_sizes:
        widow_h = int((window_size*5)/60)
        for sensor in range(1,6):
            df[f'Redox_Avg({sensor})_sigma_b_{widow_h}'] = np.nan
            df[f'Redox_Avg({sensor})_sigma_f_{widow_h}'] = np.nan
        
        backward_sigma = np.array([calculate_backward_sigma(rolling_window, breaks = breaks, window_size = window_size, starting_index = df.index[0]) for rolling_window in df.rolling(window_size)])

        df[[f'Redox_Avg(1)_sigma_b_{widow_h}', f'Redox_Avg(2)_sigma_b_{widow_h}', f'Redox_Avg(3)_sigma_b_{widow_h}', f'Redox_Avg(4)_sigma_b_{widow_h}', f'Redox_Avg(5)_sigma_b_{widow_h}']] = backward_sigma
        
        forward_sigma = df.loc[:, [f'Redox_Avg(1)_sigma_b_{widow_h}', f'Redox_Avg(2)_sigma_b_{widow_h}', f'Redox_Avg(3)_sigma_b_{widow_h}', f'Redox_Avg(4)_sigma_b_{widow_h}', f'Redox_Avg(5)_sigma_b_{widow_h}']].shift(-(window_size-1))
        df[[f'Redox_Avg(1)_sigma_f_{widow_h}', f'Redox_Avg(2)_sigma_f_{widow_h}', f'Redox_Avg(3)_sigma_f_{widow_h}', f'Redox_Avg(4)_sigma_f_{widow_h}', f'Redox_Avg(5)_sigma_f_{widow_h}']] = forward_sigma.to_numpy()

        if print_sigma_statistics == True:
            print(f"Standard deviation statistics are calculated for the window size {window_size}")
            print(df[f'Redox_Avg(1)_sigma_b_{widow_h}', f'Redox_Avg(2)_sigma_b_{widow_h}', f'Redox_Avg(3)_sigma_b_{widow_h}', f'Redox_Avg(4)_sigma_b_{widow_h}', f'Redox_Avg(5)_sigma_b_{widow_h}'].apply([min, max, np.mean, np.median], axis = 0).to_string())

    sigma_col_names_list = [col_name for col_name in df.columns.array if 'sigma' in col_name]
    removed_rows = df.index[np.any(df.loc[:, sigma_col_names_list].isna(), axis=1)]

    df.drop(removed_rows, axis=0, inplace=True)

    print(f"{len(removed_rows)} rows had to be removed due to \" hitting \" time gaps or margin areas when calculating standard deviation")
    
    return df

def merge_raw_cleaned(raw_dfs: dict[str, pd.DataFrame], clean_dfs: dict[str, pd.DataFrame], df_pairs: list[(str, str)] = df_pairs):
    merged_dfs = []
    for pair in df_pairs:
        raw = raw_dfs[pair[0]]
        if pair[1] is None:
            for x in range(1,6):
                raw[f'Redox_error_flag({x})'] = False
            raw['Redox_error_flag_available'] = False
            merged_dfs.append(raw)
        else:
            cleaned = clean_dfs[pair[1]]
            for x in range(1,6):
                cleaned[f'Redox_error_flag({x})'] = ((cleaned['Redox_error_flag'] == True) & (cleaned[f'Redox_Avg({x})'].isna() == False))
            # cleaned = cleaned.drop(['Redox_error_flag'], axis=1)
            merged = raw.merge(
                cleaned[['TIMESTAMP', 'Redox_error_flag(1)', 'Redox_error_flag(2)', 'Redox_error_flag(3)', 'Redox_error_flag(4)', 'Redox_error_flag(5)', 'Redox_error_flag']],
                how='left',
                left_on='TIMESTAMP',
                right_on='TIMESTAMP'
            )
            merged['Redox_error_flag_available'] = True
            merged_dfs.append(merged)
    return merged_dfs

def add_redox_log_cols(df: pd.DataFrame) -> pd.DataFrame:
    for x in range(1,6):
        df[f'log_redox({x})'] = np.log(df[f'Redox_Avg({x})'])
    return df

def load_data(file_names: list[str], data_folder: str) -> dict[str, pd.DataFrame]:
    """
    Load data from given file names
    :param file_names: list of file names
    :param data_folder: folder name where data is located
    :return: dictionary of DataFrames
    """
    dfs = dict()
    report_df = pd.DataFrame()

    for file_name in file_names:
        df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])
        df.rename(mapper=remove_pit_suffix, axis='columns', inplace=True)
        
        if print_text_result: print(f'===== {file_name} =====')
        
        diff_dict = dict()
        prune_unnecessary_features(df)
        
        diff_dict['dates_removed'] = filter_dates(df, file_name)
        diff_dict['duplicates_removed'] = filter_duplicate_date(df)
        
        # here is where the wavelet coefficients are calculated 
        df = fill_na_redox_values(df, file_name)
        df = calculate_wavelet_coefficients(df, 1/2, 5, file_name)
        
        window_sizes = get_time_windows([12, 24])
        
        td = timedelta(days = 0, hours = 0, minutes = 5, seconds = 0, milliseconds= 0, weeks = 0)
        
        if 'VII' not in file_name:
            df = sigma_feature_engineering(df, window_sizes=window_sizes, td=td)
        
        diff_dict['missing_data_removed'] = filter_missing_values(df, file_name)

        # instead of bundling the filtering functions they are applied one by one as the wavelet calculations wedged in 
        # stats = filter_df(df, file_name)

        # TODO add log scale to redox and error flag for each redox depth
        df = fix_types(df, file_name)
        df = add_pit_column(df, file_name)
        # df = add_redox_log_cols(df)
        report_df = pd.concat([report_df, pd.DataFrame(diff_dict, index=[file_name])])
        dfs[file_name] = df

        if print_text_result: print('\n')

    report_df = report_df.assign(Total = lambda x: (x.sum(axis=1)))
    print(report_df)

    return dfs

### Load data

In [9]:
raw_data = load_data(raw_data_names, data_folder)
clean_data = load_data(clean_data_names, data_folder)

/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])


the observations having gaps
   Observation index  Time delta (min)
0              47113              1445
1148 rows had to be removed due to " hitting " time gaps or margin areas when calculating standard deviation


/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])


there are no gaps
574 rows had to be removed due to " hitting " time gaps or margin areas when calculating standard deviation


/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])


...

/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (19,20,21,30,31,34,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])


the observations having gaps
   Observation index  Time delta (min)
0              39102                10
1              69832                60
1722 rows had to be removed due to " hitting " time gaps or margin areas when calculating standard deviation
                dates_removed  duplicates_removed  missing_data_removed  Total
pit1_data-2022              1                  12                     0     13
pit2_data-2022              1                  24                  4057   4082
pit3_data-2022              1                  12                     0     13
pit4_data-2022              1                  12                     0     13
pit1_data-2023            287                  15                     0    302
pit2_data-2023           3248               25552                  6868  35668
pit3_data-2023            287                  88                     0    375
pit4_data-2023            287                 196                     4    487


/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (37,38,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])
/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])
/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (37,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])


               dates_removed  duplicates_removed  missing_data_removed  Total
VII_PIT1_2022              1                 192                     0    193
VII_PIT2_2022              1                  24                     0     25
VII_PIT3_2022              1                  12                     0     13
VII_PIT4_2022              1                  12                     0     13


/var/folders/b2/nb86kq1j7zx8sqjthk_hy6vc0000gn/T/ipykernel_76761/1859660079.py:223: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_folder+file_name+'.csv', parse_dates=['TIMESTAMP'])


### Check data

In [10]:
raw_data['pit1_data-2022'].head()

,TIMESTAMP,Redox_Avg(1),Redox_Avg(2),Redox_Avg(3),Redox_Avg(4),Redox_Avg(5),Temp_T12_Avg(1),EC_Avg(1),Temp_T12_Avg(2),EC_Avg(2),...,Redox_Avg(1)_sigma_f_24,Redox_Avg(2)_sigma_b_24,Redox_Avg(2)_sigma_f_24,Redox_Avg(3)_sigma_b_24,Redox_Avg(3)_sigma_f_24,Redox_Avg(4)_sigma_b_24,Redox_Avg(4)_sigma_f_24,Redox_Avg(5)_sigma_b_24,Redox_Avg(5)_sigma_f_24,pit_number
287,2022-04-13 08:55:00,134,298,160,77,56,0.2,84,0.3,392,...,3.165684,1.099647,1.796422,4.505482,5.023514,2.651380,3.425405,1.620081,2.512996,1
288,2022-04-13 09:00:00,134,298,160,77,57,0.2,84,0.3,392,...,3.171434,1.098864,1.795783,4.504725,5.022991,2.653856,3.420524,1.620306,2.514988,1
289,2022-04-13 09:05:00,133,298,160,76,57,0.2,84,0.3,392,...,3.177007,1.098478,1.794928,4.504939,5.020418,2.656595,3.415380,1.620866,2.516280,1
290,2022-04-13 09:10:00,133,298,160,77,56,0.2,84,0.3,392,...,3.184965,1.097437,1.794410,4.502925,5.019582,2.657446,3.411222,1.622338,2.516780,1
291,2022-04-13 09:15:00,133,298,160,77,56,0.2,84,0.3,392,...,3.191444,1.096121,1.793165,4.503148,5.016432,2.659561,3.405667,1.623102,2.518526,1


### Dtypes in each dataframe

    NOTE: Cleaned data dtypes not changed yet. Need to think how to deal with missing values for column which should be converted to int64 from original float64

In [11]:
i = 1
print('\t\t\t'+'1\t\t'+'\t'.join(str(a) for a in [*range(2,49)]))
for t1 in zip([*raw_data.items(), *clean_data.items()]):
    print(f'{t1[0][0]}\t{i}\t'+'\t'.join(str(x) for x in t1[0][1].dtypes.array))
    i +=1

			1		2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48
pit1_data-2022	1	datetime64[ns]	int64	int64	int64	int64	int64	float64	int64	float64	int64	float64	int64	float64	int64	float64	int64	float64	float64	int64	float64	float64	float64	float64	int64	float64	float64	float64	float64	float64	bool	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	timedelta64[ns]	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	float64	int64
pit2_data-2022	2	datetime64[ns]	int64	int64	int64	int64	int64	float64	int64	float64	int64	

### Remove timestamps from raw data that do not match in cleaned data

In [12]:
for pair in df_pairs:
    raw_data_name, clean_data_name = pair[0], pair[1]
    if clean_data_name:
        print(f'Checking {raw_data_name} vs {clean_data_name}')
        clean_timestamps = clean_data[clean_data_name]['TIMESTAMP'].to_numpy()
        prev_row_count = len(raw_data[raw_data_name])
        raw_data[raw_data_name] = raw_data[raw_data_name].loc[raw_data[raw_data_name]['TIMESTAMP'].isin(clean_timestamps) == True]
        print(f'\t Rows removed {prev_row_count-len(raw_data[raw_data_name])}')

Checking pit1_data-2022 vs VII_PIT1_2022
	 Rows removed 0
Checking pit2_data-2022 vs VII_PIT2_2022
	 Rows removed 0
Checking pit3_data-2022 vs VII_PIT3_2022
	 Rows removed 0
Checking pit4_data-2022 vs VII_PIT4_2022
	 Rows removed 1442


### Add redox error flag columns and merge them with raw data

In [13]:
merged_dfs = merge_raw_cleaned(raw_data, clean_data, df_pairs)

### Combine all raw data

In [14]:
all_raw_data_df = pd.DataFrame()
for df in merged_dfs:
    all_raw_data_df = pd.concat([all_raw_data_df, df], ignore_index=True)
all_raw_data_df['Redox_error_flag'] = all_raw_data_df['Redox_error_flag'].fillna(False)

### Save data to a CSV file

In [15]:
training_folder_path = f'{data_folder}/Training/'
all_raw_data_df.to_csv(f'{training_folder_path}Raw_training_data_full.csv', index=False)